<a href="https://colab.research.google.com/github/son50git/PKT_018_TF_Text/blob/master/PKT_001_Rework_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [156]:
!pip install keras 
!pip install tensorflow

In [157]:
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb


In [158]:
# set parameters:
max_features = 5000  # ranking 5000 most frequently used words
maxlen = 400 # in a line, up to 400 words
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 3

In [159]:
import numpy as np 
# np_load_old = np.load   # save old function for calling later 

# modify the default parameters of np.load
# np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

## np.load = np_load_old

In [160]:
# note on the dataset:
# This is a dataset of 25,000 movies reviews from IMDB, labeled by sentiment (positive/negative). 
# Reviews have been preprocessed, and each review is encoded as a list of word indexes (integers). 
# For convenience, words are indexed by overall frequency in the dataset, so that 
# for instance the integer "3" encodes the 3rd most frequent word in the data. 
# This allows for quick filtering operations such as: "only consider the top 10,000 most common words, 
# but eliminate the top 20 most common words".

In [161]:
print('Loading data...')

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

Loading data...
25000 train sequences
25000 test sequences


In [162]:
print(x_train.shape)
print(type(x_train))
print(x_train[:5])
print(max(x_train[0])) # 2223 for 3000 # 4613 for 5000
print(len(x_train[0])) # 218  # 218
print(max(x_train[1])) # 2637 # 4901
print(len(x_train[1])) # 189  # 189



(25000,)
<class 'numpy.ndarray'>
[list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32])
 list([1, 194, 1153, 194, 2, 78, 22

In [163]:
# keras.datasets.imdb.get_word_index()
# {'fawn': 34701,'tsukino': 52006,'nunnery': 52007,

In [164]:
def get_fixed_word_to_id_dict(): 
    INDEX_FROM=3   # word index offset
    word_to_id = keras.datasets.imdb.get_word_index()
    word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}
    word_to_id[" "] = 0
    word_to_id["<START>"] = 1
    word_to_id["<UNK>"] = 2
    return word_to_id

In [165]:
# tmp = get_fixed_word_to_id_dict()
# {'fawn': 34704, 'tsukino': 52009,'nunnery': 52010,

In [166]:
def decode_to_sentence(data_point): 
    NUM_WORDS=1000 # only use top 1000 words
    
    word_to_id = get_fixed_word_to_id_dict()

    id_to_word = {value:key for key,value in word_to_id.items()}
    return ' '.join(id_to_word[id] for id in data_point )

In [167]:
print((x_train[0]))
print(decode_to_sentence(x_train[0]))

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]
<START> this film was just brilliant casting location scenery story directi

In [168]:
print(y_train[0]) # to see the actual sentiment

1


In [169]:
def encode_sentence(sent): 
    # print(sent)
    encoded = []
    
    word_to_id = get_fixed_word_to_id_dict() 
    
    for w in sent.split(" "): 
        if w in word_to_id: 
            encoded.append(word_to_id[w])
        else: 
            encoded.append(2)        # We used '2' for <UNK> 
    return encoded

In [170]:
words = "fawn sonja vani made-up-word"
print(encode_sentence(words))
print(encode_sentence("this does not look good"))
print(encode_sentence("hi my name is inkyu")) # 2 for "inkyu"
print(encode_sentence("Hi my name is john"))  # 2 for "Hi"

[34704, 16819, 63954, 2]
[14, 127, 24, 168, 52]
[6596, 61, 403, 9, 2]
[2, 61, 403, 9, 308]


In [171]:
len(x_train[0])

218

In [172]:
x_train[0][0:10]

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

In [173]:
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)  # right-anchored values # last maxlen words
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)


Pad sequences (samples x time)


In [174]:
print(len(x_train[0]))
x_train[0]

400


array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [175]:
print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions

# set parameters:
# max_features = 5000   
# maxlen = 400  
# batch_size = 32
# embedding_dims = 50
# filters = 250
# kernel_size = 3
# hidden_dims = 250
# epochs = 3

# from keras.layers.Embedding
# input_dim: Integer. Size of the vocabulary, i.e. maximum integer index + 1.
# output_dim: Integer. Dimension of the dense embedding.

model.add(Embedding(max_features, embedding_dims, input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters, kernel_size, padding='valid', activation='relu', strides=1))

# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

Build model...
Epoch 1/3
782/782 [==============================] - 81s 104ms/step - loss: 0.4123 - accuracy: 0.7950
Epoch 2/3
782/782 [==============================] - 83s 106ms/step - loss: 0.2337 - accuracy: 0.9050
Epoch 3/3
782/782 [==============================] - 81s 103ms/step - loss: 0.1658 - accuracy: 0.9376


In [176]:
predictions = model.predict(x_test)
print(x_test.shape)
print(type(predictions))
print(len(predictions))
print(predictions[0:10])


(25000, 400)
<class 'numpy.ndarray'>
25000
[[0.00387585]
 [0.9969244 ]
 [0.919523  ]
 [0.36218083]
 [0.9993642 ]
 [0.9507185 ]
 [0.92110133]
 [0.00380114]
 [0.98719203]
 [0.9934335 ]]


In [177]:
sentiment = ['NEG' if i < 0.5 else 'POS' for i in predictions]
sentiment[0:10]

['NEG', 'POS', 'POS', 'NEG', 'POS', 'POS', 'POS', 'NEG', 'POS', 'POS']

In [178]:
data_point_to_show = 1
print(decode_to_sentence(x_test[data_point_to_show]), "--", sentiment[data_point_to_show])

                                                                                                                                                                                                                                                                                        <START> this film requires a lot of <UNK> because it focuses on mood and character development the plot is very simple and many of the scenes take place on the same set in <UNK> <UNK> the <UNK> dennis character apartment but the film builds to a disturbing climax br br the characters create an atmosphere <UNK> with sexual tension and psychological <UNK> it's very interesting that robert <UNK> directed this considering the style and structure of his other films still the <UNK> <UNK> audio style is evident here and there i think what really makes this film work is the brilliant performance by <UNK> dennis it's definitely one of her darker characters but she plays it so perfectly and convincingly that it's scary m

In [179]:
data_point_to_show = 100
print(decode_to_sentence(x_test[data_point_to_show]), "--", sentiment[data_point_to_show])

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  <START> a quick <UNK> at the premise of this film would seem to <UNK> just another dumb <UNK> <UNK> <UNK> <UNK> fest the type where sex <UNK> death and the actors are all annoying stereotypes you actually want to die however delivers <UNK> more br br rather than focus on bare flesh and gore though there is a little of each no sex however the flick focuses on delivering <UNK> <UNK> <UNK> tension <UNK> a lovely <UNK> backdrop these feelings are further <UNK> by a cast

In [180]:
test_sentences = [] 

test_sentence = "i do not like this at all"
test_sentence = encode_sentence(test_sentence)
test_sentences.append(test_sentence) 


test_sentence = "loved it"
test_sentence = encode_sentence(test_sentence)
test_sentences.append(test_sentence) 


test_sentence = "did not love it"
test_sentence = encode_sentence(test_sentence)
test_sentences.append(test_sentence)

test_sentences

[[13, 81, 24, 40, 14, 33, 32], [447, 12], [122, 24, 119, 12]]

In [181]:
test_sentences = sequence.pad_sequences(test_sentences, maxlen=maxlen)
test_sentences

array([[  0,   0,   0, ...,  14,  33,  32],
       [  0,   0,   0, ...,   0, 447,  12],
       [  0,   0,   0, ...,  24, 119,  12]], dtype=int32)

In [182]:
predictions = model.predict(test_sentences)
sentiment = ['NEG' if i < 0.5 else 'POS' for i in predictions]

In [183]:
for i in range(test_sentences.shape[0]): 
    print(decode_to_sentence(test_sentences[i]), "--", sentiment[i])

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  i do not like this at all -- NEG
                                                                                                                                                                                     